### Frame extraction and procession

In [ ]:
# !mkdir -p /kaggle/working/frames_output/images
import cv2
import os

def extract_frames(video_path, output_folder, fps=5):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Get the original video frame rate
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Calculate the interval for frame extraction
    frame_interval = int(original_fps // fps)
    
    # Create output directory if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    # Frame counter
    frame_count = 0
    saved_frame_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # If no frame is returned, the video is over
        if not ret:
            break
        
        # Save the frame only if it's the nth frame according to the frame interval
        if frame_count % frame_interval == 0:
            output_file_path = os.path.join(output_folder, f"frame_{saved_frame_count}.jpg")
            cv2.imwrite(output_file_path, frame)
            print(f"Saved {output_file_path}")
            saved_frame_count += 1
        
        frame_count += 1
    
    # Release the video capture object
    cap.release()
    print(f"Total frames saved: {saved_frame_count}")

# Example usage
video_path = '/kaggle/input/behria3/behria.mp4'  # Path to the video file
output_folder = '/kaggle/working/frames_output'  # Folder to save extracted frames
extract_frames(video_path, output_folder, fps=5)


In [ ]:
import cv2
import glob

input_folder = "/kaggle/working/frames_output/"
output_folder = "/kaggle/working/frames_output/images/"
scale_percent = 50  # Resize to 50% of original size

for img_path in glob.glob(input_folder + "*.jpg"):
    img = cv2.imread(img_path)
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    resized = cv2.resize(img, (width, height), interpolation=cv2.INTER_AREA)
    cv2.imwrite(output_folder + img_path.split("/")[-1], resized)


In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt

def sharpness_detection(directory):
    filenames = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.jpg')]
    sharpness_dict = {}

    # Calculate the sharpness of each image
    for fname in filenames:
        image = cv2.imread(fname)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        laplacian = cv2.Laplacian(gray, cv2.CV_64F)
        var = laplacian.var()
        sharpness_dict[fname] = var

    # Sort images by sharpness
    sorted_sharpness = {k: v for k, v in sorted(sharpness_dict.items(), key=lambda item: item[1])}
    
    # Calculate mean and standard deviation of variance values
    mean = np.mean(list(sorted_sharpness.values()))
    std = np.std(list(sorted_sharpness.values()))
    
    # Display variance values and their distribution
    print("Variance of Laplace -> Mean:", mean, "Std:", std)
    fig, ax = plt.subplots()
    n, bins, patches = ax.hist(list(sorted_sharpness.values()), bins=10, density=True)
    y = (1 / (np.sqrt(2 * np.pi) * std)) * np.exp(-0.5 * (1 / std * (bins - mean))**2)
    ax.plot(bins, y, '--')
    ax.set_xlabel("Variance Value")
    ax.set_ylabel("Frequency")
    ax.set_title('Histogram of Variance Values with Normal Distribution Curve')
    plt.show()

    # Determine threshold for blurriness
    threshold = mean - std  # example threshold

    # Identify blurry images without removing two consecutive ones
    prev_removed = False
    to_remove = []
    for fname, sharpness in sorted_sharpness.items():
        if sharpness < threshold and not prev_removed:
            to_remove.append(fname)
            prev_removed = True
        else:
            prev_removed = False

    # Remove identified blurry images
    for fname in to_remove:
        os.remove(fname)
        print(f"Removed {fname}")

    return to_remove

# Example usage
directory = '/kaggle/working/frames_output/images/'
removed_files = sharpness_detection(directory)


### Feature Extraction

In [ ]:
!sudo apt-get install -y \
    git \
    cmake \
    ninja-build \
    build-essential \
    libboost-program-options-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libeigen3-dev \
    libflann-dev \
    libfreeimage-dev \
    libmetis-dev \
    libgoogle-glog-dev \
    libgtest-dev \
    libgmock-dev \
    libsqlite3-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libceres-dev

In [ ]:
# !colmap feature_extractor --help | grep CUDA


In [ ]:
# !nvidia-smi

# # Step 2: Check CUDA Version
# !nvcc --version || echo "nvcc not found"

In [ ]:
!sudo apt update --fix-missing
!sudo apt install -y colmap
!pip install pycolmap

In [ ]:
!export QT_QPA_PLATFORM='offscreen'

!sudo apt-get install -y xvfb


In [ ]:
# Set XDG_RUNTIME_DIR to a temporary directory
import os
os.environ['XDG_RUNTIME_DIR'] = '/tmp/runtime-root'


In [ ]:
# Feature Extraction with GPU
!xvfb-run colmap feature_extractor \
    --database_path "/kaggle/working/colmap_output/database.db" \
    --image_path "/kaggle/working/frames_output/images" \
    --ImageReader.camera_model PINHOLE \
    --ImageReader.single_camera 1 \
    --SiftExtraction.use_gpu 1 \
    --SiftExtraction.gpu_index 0  # Specify GPU index if multiple GPUs are available

# Exhaustive Matching with GPU
!xvfb-run colmap exhaustive_matcher \
    --database_path /kaggle/working/colmap_output/database.db \
    --SiftMatching.use_gpu 1 \
    --SiftMatching.gpu_index 0  # Specify GPU index if multiple GPUs are available


In [ ]:
!colmap mapper \
--database_path "/kaggle/working/colmap_output/database.db" \
--image_path "/kaggle/working/frames_output/input" \
--output_path "/kaggle/working/colmap_output/sparse"
  

# Gaussian splat

In [ ]:
%cd /kaggle/working/
!git clone --recursive https://github.com/camenduru/gaussian-splatting
!pip install -q plyfile

%cd /kaggle/working//gaussian-splatting
!pip install -q /kaggle/working/gaussian-splatting/submodules/diff-gaussian-rasterization
!pip install -q /kaggle/working/gaussian-splatting/submodules/simple-knn

In [ ]:
!sudo apt-get install -y \
    git \
    cmake \
    ninja-build \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libboost-regex-dev \
    libboost-test-dev \
    libeigen3-dev \
    libflann-dev \
    libfreeimage-dev \
    libmetis-dev \
    libgoogle-glog-dev \
    libgtest-dev \
    libsqlite3-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libceres-dev \
    libsuitesparse-dev \
    libgflags-dev \
    libpng-dev \
    libjpeg-dev \
    libtiff-dev \
    libxxf86vm1 \
    libxxf86vm-dev \
    libxi-dev \
    libxrandr-dev

In [ ]:
%cd /kaggle/working/gaussian-splatting
!python train.py -s /kaggle/working/frames_output


# Mesh

In [ ]:
%cd /kaggle/working/
!git clone https://github.com/Anttwo/SuGaR.git

In [ ]:
%cd /kaggle/working/SuGaR
import sys
sys.path.append('/kaggle/working/SuGaR')
sys.path.append('/kaggle/working/SuGaR/gaussian_splatting')
sys.path.append('/kaggle/working/SuGaR/gaussian_splatting/scene')
sys.path.append('/kaggle/working/SuGaR/sugar_utils')
sys.path.append('/kaggle/working/SuGaR/gaussian_splatting/submodules/simple-knn')
sys.path.append('/kaggle/workingSuGaR/gaussian_splatting/submodules/diff-gaussian-rasterization')

In [ ]:
!pip install open3d
!pip install torch torchvision torchaudio
!pip install git+https://github.com/facebookresearch/pytorch3d.git@stable

# Install Plotly
!pip install plotly

# Install Rich
!pip install rich

# Install Plyfile
!pip install plyfile==0.8.1

# Install JupyterLab
!pip install jupyterlab

# Install Node.js
!apt-get install -y nodejs

# Install IPywidgets
!pip install ipywidgets

# Install PyMCubes
!pip install --upgrade PyMCubes


In [ ]:
%cd /kaggle/working/SuGaR/gaussian_splatting/submodules/diff-gaussian-rasterization
!pip install -e .

In [ ]:
%cd /kaggle/working/SuGaR/gaussian_splatting/submodules/simple-knn
!pip install -e .

In [ ]:
%cd /kaggle/working/SuGaR
!python train_full_pipeline.py \
    -s /kaggle/working/frames_output \
    -r "dn_consistency" \
    --high_poly True \
    --export_obj True \
    --gs_output_dir "/kaggle/working/gaussian-splatting/output/9e49cfd6-b" \
    --gpu 0 \
    --white_background True
